**Install jiant and torch**


In [ ]:
!python --version

In [ ]:
!git clone https://github.com/nyu-mll/jiant.git

In [ ]:
!pip install torch==1.9.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# Remove numpy from requirements-no-torch.txt before installing the requirements
with open("jiant/requirements-no-torch.txt", "r") as f:
    lines = f.readlines()
with open("jiant/requirements-no-torch.txt", "w") as f:
    for line in lines:
        if not line.strip("\n").startswith("numpy"):
            f.write(line)

In [ ]:
!pip install -r jiant/requirements-no-torch.txt

In [ ]:
import sys
sys.path.insert(0, "/content/jiant")
from ipywidgets import IntProgress

In [ ]:
import os

import jiant.utils.python.io as py_io
import jiant.proj.simple.runscript as simple_run
import jiant.scripts.download_data.runscript as downloader


**Add Google Drive and Google Sheets credentials**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)


#### Define the parameters

In [ ]:
TASK_NAME = "copa"
HF_PRETRAINED_MODEL_NAME = "microsoft/mdeberta-v3-base"
RUN_NAME = "15"
MODEL_NAME = HF_PRETRAINED_MODEL_NAME.split("/")[-1]
EXP_DIR = "content/exp"
DATA_DIR = "content/exp/tasks"
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 16
EVAL_STEPS = 2000
EPOCHS = 10
LEARNING_RATE = 1e-5
ADAM_EPSILON=1e-8
MAX_GRAD_NORM=1.2

**Download the model and copy the Macedonian data**

In [ ]:
# Use the downloader (it's makes the creation of config files easier)
downloader.download_data([TASK_NAME], DATA_DIR)

In [ ]:
# Run this line when you run the script for the first time after connecting to the runtime in colab
# It deletes the original EN data and uploads the MK data
!rm -r /content/content/exp/tasks/data/copa/*

In [ ]:
# Run this when you start another training session 
# !rm -r /content/content/exp/runs/*
# !rm -r /content/content/exp/run_configs/*

In [ ]:
# Copy the Macedonian data 
!cp -r /gdrive/MyDrive/COPA/* /content/content/exp/tasks/data/copa 

In [ ]:
args = simple_run.RunConfiguration(
    run_name=RUN_NAME,
    exp_dir=EXP_DIR,
    data_dir=DATA_DIR,
    hf_pretrained_model_name_or_path=HF_PRETRAINED_MODEL_NAME,
    tasks=TASK_NAME,
    train_batch_size=train_batch_size,
    num_train_epochs=num_train_epochs,
#     model_weights_path = MODEL_WEIGHTS_PATH,
#     model_cache_path = MODEL_CACHE_PATH,
    eval_every_steps=2000,
    save_every_steps=2000,
    do_save=True,
    do_save_last=True,
    do_save_best=True,
    save_checkpoint_every_steps=2000,
    keep_checkpoint_when_done=True,
    write_val_preds=True,
    write_test_preds=True,
    learning_rate=learning_rate,
    adam_epsilon=adam_epsilon,
    max_grad_norm=max_grad_norm
)



**Run the model**

In [ ]:
simple_run.run_simple(args)

**Write the results in Google Sheets**

In [ ]:
# Sheet 1: BoolQ (0)
# Sheet 2: COPA (1)
# Sheet 3: MultiRC (2)
# Sheet 4: Multi-task (3)
# Sheet 5: Cross-lingual BoolQ (4)
# Sheet 6: Cross-lingual COPA (5)
# Sheet 7: Cross-lingual MultiRC (6)

worksheet = gc.open('QA_results').get_worksheet(1)

# Don't forget to move to the next row before writing the current results
cell_list = worksheet.range(f'A{int(RUN_NAME)+1}:L{int(RUN_NAME) +1}')

# COPA data:
values = [RUN_NAME, HF_PRETRAINED_MODEL_NAME, TRAIN_BATCH_SIZE, EPOCHS,LEARNING_RATE, 
          "AdamW", ADAM_EPSILON, MAX_GRAD_NORM,
          0.2538032293319702,  0.55, 0.55,0.55]

for idx, cell in enumerate(cell_list):
    cell.value = values[idx]
worksheet.update_cells(cell_list)

**Copy the best models to Google Drive**

In [ ]:
!cp /content/content/exp/runs/* -r /gdrive/MyDrive/BestCOPA/